In [12]:
import dtlpy as dl

if dl.token_expired():
    dl.login()

In [13]:
project = dl.projects.get(project_id='8c217b4c-53e3-4f05-9205-76bf88ec0838')
dataset = project.datasets.get(dataset_id='65a388a09d96c8a32c7f8b2c')

In [14]:
def filters_funct(crop):
    filters = dl.Filters()
    filters.resource = dl.FiltersResource.ITEM
    filters.add(field='metadata.crop', values=f'{crop}')
    return filters

def filter_grids_annot():
    filters = dl.Filters()
    filters.resource = dl.FiltersResource.ANNOTATION
    filters.add(field='label', values='AI Weed Area Tags.Tagging Grid', operator=dl.FiltersOperations.NOT_EQUAL)
    return filters

def get_image_id(item_name):

    def filter_name(item_name_def):
        filters = dl.Filters()
        filters.resource = dl.FiltersResource.ITEM
        filters.add(field='name', values=item_name_def)
        return filters


    filters = filter_name(item_name)
    item_id = [item.metadata['user']['ID'] for page in dataset.items.list(filters=filters) for item in page][0]

    return item_id

In [10]:
dataset_task_name = 'emergence_task'
crop = 'Corn'
filters_crop = filters_funct(crop)
filters_annot = filter_grids_annot()


def clone_dataset (dataset_task_name, filters_crop):

    task_dataset = dataset.clone(clone_name=dataset_task_name, filters=filters_crop, with_items_annotations=True, with_metadata=False)
    print ('task_dataset created')
    for page in task_dataset.items.list():
        
        for item in page:

            #Update item_ID
            item_name = item.name
            item_id = get_image_id(item_name)

            item.metadata['user'] = {}
            item.metadata['user']['ID'] = item_id
            item.update()

            #Delete labels (except grids)
            for annot in item.annotations.list(filters=filters_annot):
                annot.delete()

    return task_dataset


In [15]:
dataset_task_name = 'emergence_task_1'
crop = 'Corn'
filters_crop = filters_funct(crop)
filters_annot = filter_grids_annot()


clone_dataset (dataset_task_name, filters_crop)

Command Progress: 100%|██████████| 100/100 [02:48<00:00,  1.69s/it]
task_dataset created
Iterate Pages: 100%|██████████| 1/1 [09:16<00:00, 556.47s/it]


In [ ]:
def create_consensus_task (task_name_func, assignee_users_func):
    task_dataset.tasks.create(task_name=task_name_func, assignee_ids=assignee_users_func,
                        consensus_percentage=100, consensus_assignees=len(assignee_users_func))
    print ('Task Created')
